# Azure AI Agent Utils Demo

This notebook demonstrates how to use the `agent_utils.py` module for clean and efficient agent management.

## Prerequisites
- Azure AI Foundry project created
- Azure OpenAI resource connected

## What You'll Learn
- ✅ Initialize AgentManager
- ✅ Create agents with custom functions
- ✅ Execute agents with automatic function calling
- ✅ Stream responses in real-time
- ✅ Manage multiple agents and threads
- ✅ Clean up resources efficiently

## Table of Contents

1. [Setup and Installation](#setup-and-installation)
2. [Initialize Project Client and AgentManager](#initialize-project-client-and-agentmanager)
3. [Example 1: Simple Agent (No Functions)](#example-1-simple-agent-no-functions)
4. [Example 2: Agent with Function Calling](#example-2-agent-with-function-calling)
5. [Example 3: Streaming Responses](#example-3-streaming-responses)
6. [Example 4: Custom Streaming Callback](#example-4-custom-streaming-callback)
7. [Example 5: Multi-Turn Conversation](#example-5-multi-turn-conversation)
8. [Example 6: View Conversation History](#example-6-view-conversation-history)
9. [Example 7: Agent Update](#example-7-agent-update)
10. [Example 8: List All Agents](#example-8-list-all-agents)
11. [Example 9: Error Handling in Function Calls](#example-9-error-handling-in-function-calls)
12. [Cleanup: Delete All Resources](#cleanup-delete-all-resources)
13. [Summary](#summary)

## Configure PATH for Azure CLI

Ensure the Azure CLI is accessible in the notebook kernel's PATH.

In [ ]:
import os
import shutil

new_path_entry = "/opt/homebrew/bin"  # Replace with the directory you want to add
current_path = os.environ.get('PATH', '')

if new_path_entry not in current_path.split(os.pathsep):
    os.environ['PATH'] = new_path_entry + os.pathsep + current_path
    print(f"Updated PATH for this session: {os.environ['PATH']}")
else:
    print(f"PATH already contains {new_path_entry}: {current_path}")

# You can then verify with shutil.which again
print(f"Location of 'az' found by kernel now: {shutil.which('az')}")

## Setup and Installation

In [ ]:
import os
import sys
import json
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# Add utils directory to path
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'utils'))
from agent_utils import AgentManager

# Load environment variables
load_dotenv("../.env")

print("✅ Imports successful")

## Initialize Project Client and AgentManager

In [ ]:
# Get project endpoint
endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
model = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o")

if not endpoint:
    raise ValueError("Please set AZURE_AI_PROJECT_ENDPOINT in .env")

# Initialize client
project_client = AIProjectClient(
    endpoint=endpoint,
    credential=DefaultAzureCredential()
)

# Initialize AgentManager
manager = AgentManager(project_client)

print("✅ AgentManager initialized")
print(f"📦 Endpoint: {endpoint}")
print(f"🤖 Model: {model}")

## Example 1: Simple Agent (No Functions)

Create a basic conversational agent without any function calling.

In [ ]:
# Create a simple agent
simple_agent = manager.create_agent(
    model=model,
    name="SimpleAssistant",
    instructions="You are a helpful AI assistant that provides concise and accurate answers."
)

In [ ]:
# Create a thread and run the agent
simple_thread = manager.create_thread()

response = manager.run_agent_simple(
    thread_id=simple_thread.id,
    agent_id=simple_agent.id,
    user_message="What are the three laws of robotics?"
)

print("\n" + "=" * 80)
print(response)
print("=" * 80)

### Using Configuration Parameters

You can control the agent's behavior with temperature, max_tokens, etc.

In [ ]:
# Create another thread to test with different parameters
config_thread = manager.create_thread()

print("\n🤖 Agent Response (with custom config):")
print("=" * 80)
print("Config: temperature=0.3 (more focused), max_completion_tokens=150")
print("=" * 80)

response = manager.run_agent_simple(
    thread_id=config_thread.id,
    agent_id=simple_agent.id,
    user_message="Explain machine learning in simple terms.",
    temperature=0.3,  # More focused and deterministic
    max_completion_tokens=150  # Limit response length
)

print("\n" + response)
print("=" * 80)

# Compare with higher temperature
config_thread_2 = manager.create_thread()

print("\n🤖 Agent Response (higher temperature):")
print("=" * 80)
print("Config: temperature=0.9 (more creative), max_completion_tokens=150")
print("=" * 80)

response2 = manager.run_agent_simple(
    thread_id=config_thread_2.id,
    agent_id=simple_agent.id,
    user_message="Explain machine learning in simple terms.",
    temperature=0.9,  # More creative and varied
    max_completion_tokens=150
)

print("\n" + response2)
print("=" * 80)

## Example 2: Agent with Function Calling

Create an agent with custom functions for data retrieval and calculations.

In [ ]:
# Define utility functions
def get_current_time(timezone: str = "UTC") -> str:
    """Get the current time in a specific timezone"""
    from datetime import datetime
    import pytz

    try:
        tz = pytz.timezone(timezone)
        current_time = datetime.now(tz)
        return json.dumps({
            "timezone": timezone,
            "time": current_time.strftime("%Y-%m-%d %H:%M:%S"),
            "day_of_week": current_time.strftime("%A")
        })
    except Exception as e:
        return json.dumps({"error": str(e)})


def calculate_percentage(part: float, whole: float) -> str:
    """Calculate what percentage one number is of another"""
    if whole == 0:
        return json.dumps({"error": "Cannot divide by zero"})

    percentage = (part / whole) * 100
    return json.dumps({
        "part": part,
        "whole": whole,
        "percentage": round(percentage, 2)
    })


def get_word_count(text: str) -> str:
    """Count the number of words in a text"""
    words = text.split()
    return json.dumps({
        "text_length": len(text),
        "word_count": len(words),
        "character_count": len(text.replace(" ", ""))
    })


# Create functions dictionary
utility_functions = {
    "get_current_time": get_current_time,
    "calculate_percentage": calculate_percentage,
    "get_word_count": get_word_count
}

print("✅ Utility functions defined")

In [ ]:
# Create agent with functions
utility_agent = manager.create_agent(
    model=model,
    name="UtilityAgent",
    instructions="""You are a helpful utility assistant with access to tools for:
    - Getting current time in different timezones
    - Calculating percentages
    - Counting words in text
    
    Always use the appropriate function when the user asks for these operations.""",
    functions=utility_functions
)

In [ ]:
# Create thread and test function calling
utility_thread = manager.create_thread()

print("\n🤖 Agent Response (with function calling):")
print("=" * 80)

response = manager.run_agent(
    thread_id=utility_thread.id,
    agent_id=utility_agent.id,
    user_message="What time is it in America/New_York? Also, what percentage is 45 out of 200?",
    functions=utility_functions,
    verbose=True
)

print("\n" + "=" * 80)
print(response)
print("=" * 80)

## Example 3: Streaming Responses

Stream agent responses in real-time for better user experience.

In [ ]:
# Create a new thread for streaming
stream_thread = manager.create_thread()

print("\n🤖 Streaming Response:")
print("=" * 80)

response = manager.stream_agent(
    thread_id=stream_thread.id,
    agent_id=utility_agent.id,
    user_message="Explain what makes a good AI assistant in 3 bullet points."
)

print("\n" + "=" * 80)
print("✅ Streaming completed")

## Example 4: Custom Streaming Callback

Use a custom callback to process streaming chunks.

In [ ]:
# Custom callback for streaming
streamed_chunks = []

def custom_callback(chunk: str):
    """Custom callback to collect chunks"""
    streamed_chunks.append(chunk)
    print(chunk, end='', flush=True)

# Create new thread
callback_thread = manager.create_thread()

print("\n🤖 Custom Streaming with Callback:")
print("=" * 80)

response = manager.stream_agent(
    thread_id=callback_thread.id,
    agent_id=simple_agent.id,
    user_message="What is quantum computing?",
    callback=custom_callback
)

print("\n" + "=" * 80)
print("\n📊 Statistics:")
print(f"   Total chunks received: {len(streamed_chunks)}")
print(f"   Total characters: {len(response)}")

## Example 5: Multi-Turn Conversation

Have a multi-turn conversation with the agent.

In [ ]:
# Create conversation thread
conversation_thread = manager.create_thread(metadata={"type": "multi-turn"})

# First turn
print("\n👤 User: Tell me about Python programming.")
print("🤖 Assistant:")
print("=" * 80)

response1 = manager.run_agent_simple(
    thread_id=conversation_thread.id,
    agent_id=simple_agent.id,
    user_message="Tell me about Python programming.",
    verbose=False
)
print(response1)

# Second turn
print("\n" + "=" * 80)
print("\n👤 User: What are its main advantages?")
print("🤖 Assistant:")
print("=" * 80)

response2 = manager.run_agent_simple(
    thread_id=conversation_thread.id,
    agent_id=simple_agent.id,
    user_message="What are its main advantages?",
    verbose=False
)
print(response2)
print("=" * 80)

## Example 6: View Conversation History

Retrieve and format the full conversation history.

In [ ]:
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'utils'))
from agent_utils import format_messages

# Get all messages from the conversation
messages = manager.get_messages(conversation_thread.id)

print("\n📜 Conversation History:")
print(format_messages(messages))

## Example 7: Agent Update

Update an existing agent's configuration.

In [ ]:
# Update the simple agent with new instructions
updated_agent = manager.update_agent(
    agent_id=simple_agent.id,
    instructions="""You are a helpful AI assistant that provides concise answers.
    Always format your responses with clear structure and use bullet points when listing items."""
)

## Example 8: List All Agents

View all agents created in your project.

In [ ]:
# List all agents from database
agents_metadata = manager.list_agents_metadata(limit=10)

print("\n📋 Available Agents:")
print("=" * 80)

if not agents_metadata:
    print("No agents found in database.")
else:
    for i, metadata in enumerate(agents_metadata, 1):
        card_width = 78

        # Card header
        print(f"\n┌{'─' * card_width}┐")
        name = metadata.get('name', 'Unnamed Agent')
        name_padding = card_width - len(name) - 4
        print(f"│ 🤖 {name}{' ' * name_padding}│")
        print(f"├{'─' * (card_width)}┤")

        # Category
        category = metadata.get('category') or 'General'
        category_line = f"📂 Category: {category}"
        category_padding = card_width - len(category_line) - 2
        print(f"│ {category_line}{' ' * category_padding}│")

        # Description
        description = metadata.get('description') or 'No description available'
        desc_label = "📝 Description: "
        desc_max_width = card_width - len(desc_label) - 2
        desc_lines = [description[j:j+desc_max_width]
                      for j in range(0, len(description), desc_max_width)]

        # First line with label
        first_line = f"{desc_label}{desc_lines[0]}"
        first_padding = card_width - len(first_line) - 2
        print(f"│ {first_line}{' ' * first_padding}│")

        # Continuation lines
        for line in desc_lines[1:]:
            continuation_padding = card_width - len(line) - 4
            print(f"│    {line}{' ' * continuation_padding}│")

        # Functions
        has_function = metadata.get('function', False)
        function_list = metadata.get('functionList', [])
        
        # Handle function_list as either list of strings or empty
        if isinstance(function_list, str):
            function_list = []
        
        if has_function and function_list:
            # Parse format: "uuid<sep>function_name"
            func_names = []
            for func_entry in function_list[:3]:
                if isinstance(func_entry, str) and '<sep>' in func_entry:
                    func_name = func_entry.split('<sep>')[1]
                    func_names.append(func_name)
                elif isinstance(func_entry, dict):
                    func_names.append(func_entry.get('name', 'Unknown'))
            tools_display = ', '.join(func_names) if func_names else "Yes (details not available)"
        elif has_function:
            tools_display = "Yes (details not available)"
        else:
            tools_display = "None"

        tools_line = f"🛠️  Functions: {tools_display}"
        tools_padding = card_width - len(tools_line) - 2
        print(f"│ {tools_line}{' ' * tools_padding}│")

        # Status
        status = metadata.get('status', 'unknown')
        status_line = f"📊 Status: {status}"
        status_padding = card_width - len(status_line) - 2
        print(f"│ {status_line}{' ' * status_padding}│")

        # Azure Agent ID
        azure_id = metadata.get('azure_agent_id', 'N/A')
        azure_id_display = azure_id[:50] + \
            ('...' if len(azure_id) > 50 else '')
        id_line = f"🆔 Azure ID: {azure_id_display}"
        id_padding = card_width - len(id_line) - 2
        print(f"│ {id_line}{' ' * id_padding}│")

        # Database ID
        db_id = metadata.get('id', 'N/A')
        db_id_display = db_id[:50] + ('...' if len(db_id) > 50 else '')
        db_id_line = f"💾 DB ID: {db_id_display}"
        db_id_padding = card_width - len(db_id_line) - 2
        print(f"│ {db_id_line}{' ' * db_id_padding}│")

        print(f"└{'─' * card_width}┘")

print("\n" + "=" * 80)

## Example 9: Error Handling in Function Calls

See how the manager handles errors in function execution.

In [ ]:
# Create thread for error testing
error_thread = manager.create_thread()

print("\n🤖 Testing Error Handling:")
print("=" * 80)

# This will trigger the division by zero error in calculate_percentage
response = manager.run_agent(
    thread_id=error_thread.id,
    agent_id=utility_agent.id,
    user_message="What percentage is 50 out of 0?",
    functions=utility_functions,
    verbose=True
)

print("\n" + "=" * 80)
print(response)
print("=" * 80)

## Cleanup: Delete All Resources

Clean up all agents and threads created in this notebook.

In [ ]:
# Collect all agent and thread IDs
agent_ids_to_delete = [
    simple_agent.id,
    utility_agent.id
]

thread_ids_to_delete = [
    simple_thread.id,
    config_thread.id,
    config_thread_2.id,
    utility_thread.id,
    stream_thread.id,
    callback_thread.id,
    conversation_thread.id,
    error_thread.id
]

# Cleanup using the manager
result = manager.cleanup(
    agent_ids=agent_ids_to_delete,
    thread_ids=thread_ids_to_delete,
    verbose=True
)

print(f"\n✅ Cleanup summary: {result}")

## Summary

### What We Covered:

1. **AgentManager Initialization**: Set up the manager with project client
2. **Simple Agent**: Created basic conversational agent
3. **Function Calling**: Added custom functions for data retrieval and calculations
4. **Streaming**: Implemented real-time response streaming
5. **Custom Callbacks**: Used custom callbacks for streaming processing
6. **Multi-Turn Conversations**: Maintained context across multiple exchanges
7. **Conversation History**: Retrieved and formatted message history
8. **Agent Updates**: Modified agent configurations
9. **Error Handling**: Demonstrated graceful error handling in functions
10. **Batch Cleanup**: Efficiently cleaned up multiple resources

### Key Benefits of agent_utils.py:

- **Simplified API**: Clean, intuitive methods for all operations
- **Automatic Function Handling**: Functions called automatically during runs
- **Error Resilience**: Graceful error handling with informative messages
- **Progress Feedback**: Verbose mode shows execution progress
- **Flexible Cleanup**: Delete multiple resources in one call
- **Streaming Support**: Real-time responses with custom callbacks

### Next Steps:

- Create specialized agents for your use case
- Implement more complex function tools
- Build multi-agent systems with different roles
- Add tracing and monitoring
- Integrate with file search and code interpreter

### Learn More:

- [agent_utils.py documentation](./README_agent_utils.md)
- [Azure AI Foundry Documentation](https://learn.microsoft.com/azure/ai-studio/)
- [Original examples](./01_foundry_agent.ipynb)